# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846828551529                   -0.70    4.5         
  2   -7.852303652810       -2.26       -1.53    1.0   17.0ms
  3   -7.852616345845       -3.50       -2.55    1.8   19.0ms
  4   -7.852646000512       -4.53       -2.89    2.5   22.7ms
  5   -7.852646513556       -6.29       -3.20    1.0   17.0ms
  6   -7.852646678023       -6.78       -3.98    1.0   17.0ms
  7   -7.852646686148       -8.09       -5.35    1.5   19.0ms
  8   -7.852646686722       -9.24       -5.38    2.2   22.6ms
  9   -7.852646686730      -11.10       -6.55    1.0   17.2ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846908136735                   -0.70           4.8         
  2   -7.852526332064       -2.25       -1.64   0.80    2.0    201ms
  3   -7.852635891437       -3.96       -2.73   0.80    1.0   15.9ms
  4   -7.852646451741       -4.98       -3.23   0.80    2.2   20.8ms
  5   -7.852646667051       -6.67       -4.04   0.80    1.0   15.8ms
  6   -7.852646686340       -7.71       -4.71   0.80    1.8   19.3ms
  7   -7.852646686723       -9.42       -5.40   0.80    1.8   18.8ms
  8   -7.852646686729      -11.23       -6.87   0.80    1.5   17.6ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.369509e+01     3.529451e+00
 * time: 0.044946908950805664
     1     9.288682e-01     1.822550e+00
 * time: 0.21744298934936523
     2    -1.488919e+00     2.047684e+00
 * time: 0.23557686805725098
     3    -3.726736e+00     1.482730e+00
 * time: 0.2614259719848633
     4    -4.881989e+00     1.688693e+00
 * time: 0.33816099166870117
     5    -6.526090e+00     1.016655e+00
 * time: 0.3641479015350342
     6    -7.236181e+00     6.210393e-01
 * time: 0.3899669647216797
     7    -7.585957e+00     3.349985e-01
 * time: 0.40774083137512207
     8    -7.730784e+00     1.134542e-01
 * time: 0.4254019260406494
     9    -7.787688e+00     1.650540e-01
 * time: 0.4431118965148926
    10    -7.818084e+00     5.666852e-02
 * time: 0.4608299732208252
    11    -7.836251e+00     7.119890e-02
 * time: 0.47864794731140137
    12    -7.843966e+00     7.330045e-02
 * time: 0.4963228702545166
    13    -7.845234e+00     6.054465e-02
 * time: 0.514

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846844736258                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645891766                   -1.64         
  2   -7.852646686730       -6.10       -3.71    1.45s
  3   -7.852646686730      -13.25       -7.23    141ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.0501987771093666e-7
|ρ_newton - ρ_scfv| = 5.656821295476821e-7
|ρ_newton - ρ_dm|   = 1.0078171005778773e-9
